In [15]:
import numpy as np
import pandas as pd
# Paquetes

# Gurobi Python Interface
from gurobipy import*

In [16]:
# Data
scenarios = [1,2,3,4]
vartypes = ["A","B","t1","t2"]
hrs_3days = float(72)
probs = {1:0.16,2:0.24,3:0.24,4:0.36}
prices = {1:[2*25,26,27,28],2:[2*25,26,27,26],3:[2*25,26,23,24],4:[2*25,26,23,22]}

In [17]:
# Create Model
model = Model('example 2.8')

In [18]:
# Variables
P = model.addVars(scenarios,vartypes,name = "P",lb=0,vtype=GRB.CONTINUOUS)

# Model update
model.update()

In [19]:
# Objective
obj = quicksum(probs[scenario]*hrs_3days*quicksum(prices[scenario][i]*P[scenario,vartype] for i,vartype in enumerate(vartypes)) for scenario in scenarios)
model.setObjective(obj, GRB.MAXIMIZE) # maximize profit


In [20]:
# Constraints
# Power produced in first period cap consist of contract A and pool and is capped by 120 MW
model.addConstrs(P[scenario,"A"] + P[scenario,"t1"] <= 120 for scenario in scenarios)
# Power produced in first period cap consist of contract A, B and pool and is capped by 120 MW
model.addConstrs(P[scenario,"A"] + P[scenario,"B"] + P[scenario,"t2"] <= 120 for scenario in scenarios)

model.addConstrs(P[scenario,"A"] <= 50 for scenario in scenarios)

model.addConstrs(P[scenario,"B"] <= 40 for scenario in scenarios)


model.addConstr(P[1,"A"] == P[2,"A"])
model.addConstr(P[1,"A"] == P[3,"A"])
model.addConstr(P[1,"A"] == P[4,"A"])
model.addConstr(P[2,"A"] == P[3,"A"])
model.addConstr(P[2,"A"] == P[4,"A"])
model.addConstr(P[3,"A"] == P[4,"A"])

model.addConstr(P[1,"B"] == P[2,"B"])
model.addConstr(P[3,"B"] == P[4,"B"])

model.addConstr(P[1,"t1"] == P[2,"t1"])
model.addConstr(P[3,"t1"] == P[4,"t1"])

<gurobi.Constr *Awaiting Model Update*>

In [21]:
# Update model
model.update()

# Proceed to optimize
model.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 26 rows, 16 columns and 48 nonzeros
Model fingerprint: 0x70e4e241
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+02, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+01, 1e+02]
Presolve removed 26 rows and 16 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.3248960e+05   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  4.324896000e+05


In [22]:
# Visualization
rows = scenarios.copy()
columns = ["A","B","t1","t2"]
Pi_table = pd.DataFrame(columns=columns, index=rows, data=0.0)

for (scenario,vartype) in P.keys():
    Pi_table.loc[scenario,vartype] = np.round(P[scenario,vartype].x, 1)
    
Pi_table

,A,B,t1,t2
1,50.0,0.0,70.0,70.0
2,50.0,0.0,70.0,70.0
3,50.0,40.0,70.0,30.0
4,50.0,40.0,70.0,30.0
